In [1]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.transforms.functional as RF
from PIL import Image
import numpy as np
import itertools
import random,cv2,pandas,os,numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
seed = 999
print("Random Seed: ", seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True)

workers = 2
nz = 100
beta1 = 0.5
lr = 0.001
ngpu=1
ngf,nc = 3,3
ndf = 64

device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Random Seed:  999


In [3]:
test_data={}
test_adc=pandas.read_csv("/kaggle/input/ariel-data-challenge-2024/test_adc_info.csv")

def apply_linear_corr(linear_corr,clean_signal):
    linear_corr = numpy.flip(linear_corr, axis=0)
    for x, y in itertools.product(
                range(clean_signal.shape[1]), range(clean_signal.shape[2])
            ):
        poli = numpy.poly1d(linear_corr[:, x, y])
        clean_signal[:, x, y] = poli(clean_signal[:, x, y])
    return clean_signal
if os.path.exists(f"/kaggle/working/train_rafire")==False:
    os.mkdir(f"/kaggle/working/train_rafire")
    
for i in test_adc['planet_id']:
    if os.path.exists(f"/kaggle/working/train_rafire/{i}")==False:
        os.mkdir(f"/kaggle/working/train_rafire/{i}")
    print(i)
    a_rafire_exo=pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/AIRS-CH0_signal.parquet',engine='pyarrow')
    a_rafire_data=(numpy.array(a_rafire_exo).reshape((11250, 32, 356))*float(test_adc[test_adc['planet_id']==i]['FGS1_adc_gain']))+float(test_adc[test_adc['planet_id']==i]['FGS1_adc_offset'])

    a_dark_frame = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/AIRS-CH0_calibration/dark.parquet', engine='pyarrow').to_numpy()
    a_dead_frame = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/AIRS-CH0_calibration/dead.parquet', engine='pyarrow').to_numpy()
    a_flat_frame = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/AIRS-CH0_calibration/flat.parquet', engine='pyarrow').to_numpy()
    a_linear_corr = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/AIRS-CH0_calibration/linear_corr.parquet').values.astype(numpy.float64).reshape((1, 192, 356))
    a_read = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/AIRS-CH0_calibration/read.parquet').values.astype(numpy.float64)
    
    a_rafire_z=numpy.add(numpy.add(numpy.subtract(numpy.subtract(a_rafire_data,a_dark_frame),a_dead_frame),a_flat_frame),a_read)
    a_rafire_z=numpy.average(apply_linear_corr(a_linear_corr,a_rafire_z),(0))
    
    f_rafire_exo=pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/FGS1_signal.parquet',engine='pyarrow')
    f_rafire_data=(numpy.array(f_rafire_exo).reshape(135000, 32, 32)*float(test_adc[test_adc['planet_id']==i]['FGS1_adc_gain']))+float(test_adc[test_adc['planet_id']==i]['FGS1_adc_offset'])

    f_dark_frame = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/FGS1_calibration/dark.parquet', engine='pyarrow').to_numpy()
    f_dead_frame = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/FGS1_calibration/dead.parquet', engine='pyarrow').to_numpy()
    f_flat_frame = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/FGS1_calibration/flat.parquet', engine='pyarrow').to_numpy()
    f_linear_corr = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/FGS1_calibration/linear_corr.parquet').values.astype(numpy.float64).reshape((6, 32, 32))
    f_read = pandas.read_parquet(f'/kaggle/input/ariel-data-challenge-2024/test/{i}/FGS1_calibration/read.parquet').values.astype(numpy.float64)
    
    f_rafire_z=numpy.add(numpy.add(numpy.subtract(numpy.subtract(f_rafire_data,f_dark_frame),f_dead_frame),f_flat_frame),f_read)
    f_rafire_z=numpy.average(apply_linear_corr(f_linear_corr,f_rafire_z),(0))
    
    rafire_z=numpy.concatenate([f_rafire_z, a_rafire_z], axis=1).reshape(-1)
    test_data[i]=rafire_z
    
    #print(rafire_z)
    #plt.imshow(rafire_z)
    #plt.title(f'Image {i}')
    #plt.show()
    #plt.plot(rafire_z)
    #plt.title(f'Signal Plot {i}')
    #plt.show()
    
pandas.DataFrame(test_data)

499191466


/tmp/ipykernel_17/3500598906.py:20: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  a_rafire_data=(numpy.array(a_rafire_exo).reshape((11250, 32, 356))*float(test_adc[test_adc['planet_id']==i]['FGS1_adc_gain']))+float(test_adc[test_adc['planet_id']==i]['FGS1_adc_offset'])
/tmp/ipykernel_17/3500598906.py:32: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  f_rafire_data=(numpy.array(f_rafire_exo).reshape(135000, 32, 32)*float(test_adc[test_adc['planet_id']==i]['FGS1_adc_gain']))+float(test_adc[test_adc['planet_id']==i]['FGS1_adc_offset'])


,499191466
0,-90.321569
1,-90.022258
2,-88.236102
3,-89.814317
4,-88.715866
...,...
12411,-0.090294
12412,0.128904
12413,0.090263
12414,-0.015499


In [4]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire=nn.Sequential(

            nn.Linear(12416, 2024),
            nn.Dropout(p=0.3),
            nn.GELU(),
            
            nn.Linear(2024, 1524),
            nn.Dropout(p=0.3),
            nn.GELU(),
            
            nn.Linear(1524, 1124),
            nn.Dropout(p=0.35),
            nn.GELU(),
            
            nn.Linear(1124, 924),
            nn.Dropout(p=0.35),
            nn.GELU(),
            
            nn.Linear(924, 824),
            nn.Dropout(p=0.25),
            nn.GELU(),
            
            nn.Linear(824, 724),
            nn.Dropout(p=0.25),
            nn.GELU(),
            
            nn.Linear(724, 624),
            nn.Dropout(p=0.2),
            nn.GELU(),
            
            nn.Linear(624, 524),
            nn.Dropout(p=0.2),
            nn.GELU(),
            
            nn.Linear(524, 424),
            nn.Dropout(p=0.15),
            nn.GELU(),
            
            nn.Linear(424, 324),
            nn.Dropout(p=0.1),
            nn.GELU(),
            
            nn.Linear(324, 283),
            nn.SiLU()
        )

    def forward(self,x):
        return self.rafire(x)

In [5]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data, gain=0.02)
        
class Discriminator_z(nn.Module):
    def __init__(self):
        super().__init__()
        self.rafire=nn.Sequential(

            nn.Linear(283, 924),
            nn.Dropout(p=0.30),
            nn.SiLU(),
            
            nn.Linear(924, 824),
            nn.Dropout(p=0.25),
            nn.SiLU(),
            
            nn.Linear(824, 724),
            nn.Dropout(p=0.25),
            nn.SiLU(),
            
            nn.Linear(724, 624),
            nn.Dropout(p=0.2),
            nn.SiLU(),
            
            nn.Linear(624, 524),
            nn.Dropout(p=0.2),
            nn.SiLU(),
            
            nn.Linear(524, 424),
            nn.Dropout(p=0.15),
            nn.SiLU(),
            
            nn.Linear(424, 324),
            nn.Dropout(p=0.1),
            nn.SiLU(),
            
            nn.Linear(324, 283),
            nn.SiLU()
        )

    def forward(self,x):
        return self.rafire(x)

In [6]:
netD = Discriminator().to(device)
if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
netD.apply(weights_init)

netD.load_state_dict(torch.load(f"/kaggle/input/radire-model/result_rf_wl/2.638780927100015e-07.pth",map_location=torch.device('cpu'),weights_only=False))

netZ = Discriminator_z().to(device)
if (device.type == 'cuda') and (ngpu > 1):
    netZ = nn.DataParallel(netZ, list(range(ngpu)))
netZ.apply(weights_init)

netZ.load_state_dict(torch.load(f"/kaggle/input/ariel-sigma/result_rf_sigma/1.5759950144911272e-07.pth",map_location=torch.device('cpu'),weights_only=False))


<All keys matched successfully>

In [7]:
def normalize(arr, t_min, t_max):
    shape=arr.shape
    arr=arr.reshape(-1)
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return numpy.array(norm_arr).reshape(shape)

In [8]:
rafire_sub=pandas.read_csv("/kaggle/input/ariel-data-challenge-2024/sample_submission.csv").to_dict('list')

for i in rafire_sub.keys():
    rafire_sub[i]=[]
for i in test_data.keys():
    rafire_data=torch.from_numpy(test_data[i]).to(device).float()
    output = netD(rafire_data.float())#-0.0015
    output=abs(output)
    rafire_Z = abs(netZ(output.float())).cpu().detach().numpy()
    output=output.cpu().detach().numpy()
    
    rafire_sub[f'planet_id'].append(i)
    for j in zip(range(len(output)),output,rafire_Z):
        rafire_sub[f'wl_{j[0]+1}'].append(j[1])
        rafire_sub[f'sigma_{j[0]+1}'].append(j[2])

pandas.DataFrame(rafire_sub).set_index('planet_id').to_csv(f"submission.csv")
pandas.DataFrame(rafire_sub).set_index('planet_id')

,wl_1,wl_2,wl_3,wl_4,wl_5,wl_6,wl_7,wl_8,wl_9,wl_10,...,sigma_274,sigma_275,sigma_276,sigma_277,sigma_278,sigma_279,sigma_280,sigma_281,sigma_282,sigma_283
planet_id,,,,,,,,,,,,,,,,,,,,,
499191466,0.002456,0.002497,0.002485,0.002484,0.002478,0.002473,0.002481,0.002494,0.00248,0.002476,...,0.000547,0.000492,0.000712,0.000413,0.000401,0.000425,0.000549,0.000438,0.000616,0.00056
